In [ ]:
# Importing Libraries
import requests
import pandas as pd
import time
import os

# Defining indicators and their corresponding World Bank API codes
indicators = {'debt': 'GC.DOD.TOTL.GD.ZS', 
              'inflation': 'FP.CPI.TOTL.ZG', 
              'reserves': 'FI.RES.TOTL.MO', 
              'current_account': 'BN.CAB.XOKA.GD.ZS', 
              'fdi': 'BX.KLT.DINV.WD.GD.ZS', 
              'gdp_growth': 'NY.GDP.MKTP.KD.ZG'}

def scrape_indicator(indicator_code, start = 2010, end = 2022): # Defining our function to scrape data
    link = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_code}" # Defining link
    parameters = {'format': 'json', 'date': f"{start}:{end}", 'per_page': 30000} # Defining parameters

    # Making the API request
    response = requests.get(link, params = parameters)
    time.sleep(0.2)

    # Checking for a successful status code
    if response.status_code != 200:
        print(f"Error fetching {indicator_code}")
        return None

    # Converting the response from the API into a python dictionary
    json_data = response.json()

    # Checking the data exists and has been correctly converted
    if len(json_data) < 2:
        print(f"No data found for {indicator_code}")
        return None

    # Extracting the data from the response and storing it in a list of dictionaries
    data = []
    for entry in json_data[1]:
        if entry['value'] is not None:
            data.append({
                "country": entry['country']['value'],
                "iso": entry['country']['id'],
                "year": int(entry['date']),
                "value": entry['value']
            })
            
    return pd.DataFrame(data)

In [ ]:
# Calling the function for each indicator
for name, code in indicators.items():
    df = scrape_indicator(code, start = 2010, end = 2022)

    directory = '../data'
    
    # Saving the data as a CSV file
    if df is not None:
        path = os.path.join(directory, f"{name}_data.csv")
        df.to_csv(path, index = False)
        print(f"Saved {name} data to {path}")
    else:
        print(f"No data found for {name}")

In [ ]:
# Scraping UN geoscheme country-region classifications
def scrape_region(link):
    tables = pd.read_html(link)

    # Identifying and copying relevant columns
    main_table = tables[0]
    region_dataframe = main_table[['Country or Area', 'Geographical subregion']].copy()
    region_dataframe.columns = ['country', 'region'] # Renaming relevant columns for compatibility with WorldBank data

    return region_dataframe

# Scrape UN geoscheme data and save it as a CSV file
region_dataframe = scrape_region('https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_the_United_Nations_geoscheme')
region_dataframe.to_csv('../data/country_regions.csv', index = False)